# Data Wrangling for Customer File

## Table of Contents
* [Importing Libraries](#Importing-Libraries)
* [Gathering Information about Customer File](#Gathering-Information-about-Customer-File)
* [Data Wrangling for Customer File](#Data-Wrangling-for-Customer-File)
    * [Changing column names](#Changing-column-names)
* [Data Consistency for Customer File](#Data-Consistency-for_Customer-File)
    * [Checking for mixed data types, missing values, and duplicates](#Checking-for-mixed-data-types,-missing-values,-and-duplicates)
* [Combining Orders_Products & Customer Data Sets](#Combining-Orders_Products-&-Customer-Data-Sets)
* [Exporting Merged File](#Exporting-Merged-File)

# Importing Libraries

In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import os

# Import Files

In [2]:
# Importing customer CSV file
df_customer = pd.read_csv(r'/Users/suzandiab/Documents/Instacart Basket Analysis/02 Data/Original Data/customers.csv', 
                      index_col = False)

In [3]:
# Importing Orders_Products CSV file
ords_prods_merged = pd.read_pickle(r'/Users/suzandiab/Documents/Instacart Basket Analysis/02 Data/Prepared Data/ords_prods_merge_flagged.pkl')

# Gathering Information about Customer File

In [4]:
# Checking dimensions
df_customer.shape

(206209, 10)

In [5]:
# Checking first 5 rows 
df_customer.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [6]:
# Data types for customer df
df_customer.dtypes

user_id          int64
First Name      object
Surnam          object
Gender          object
STATE           object
Age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

In [7]:
# Descriptive statistics of customer df 
df_customer.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


# Data Wrangling for Customer File

## Changing column names

In [8]:
# Changing data type of user id to string so describe function can ignore 
df_customer['user_id'] = df_customer['user_id'].astype('str')

In [9]:
# Checking data type of user id column, string
df_customer['user_id'].dtype

dtype('O')

In [10]:
# Re-checking descriptive statistics of customer df without user_id
df_customer.describe()

,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000
mean,49.501646,1.499823,94632.852548
std,18.480962,1.118433,42473.786988
min,18.000000,0.000000,25903.000000
25%,33.000000,0.000000,59874.000000
50%,49.000000,1.000000,93547.000000
75%,66.000000,3.000000,124244.000000
max,81.000000,3.000000,593901.000000


In [11]:
# Checking values of date joined column
df_customer['date_joined'].value_counts(dropna = False)

date_joined
9/17/2018     213
2/10/2018     212
4/1/2019      211
9/21/2019     211
12/19/2017    210
             ... 
9/1/2018      141
1/22/2018     140
11/24/2017    139
7/18/2019     138
8/6/2018      128
Name: count, Length: 1187, dtype: int64

I would not drop any columns in this df, as all of them seem important to the analysis questions.

In [12]:
# Checking values of fam status column
df_customer['fam_status'].value_counts(dropna = False)

fam_status
married                             144906
single                               33962
divorced/widowed                     17640
living with parents and siblings      9701
Name: count, dtype: int64

In [13]:
# Renaming Surnam column
df_customer.rename(columns = {'Surnam' : 'last_name'}, inplace = True)

In [14]:
# Renaming n_dependants column
df_customer.rename(columns = {'n_dependants' : 'number_of_dependents'}, inplace = True)

In [15]:
# Renaming fam_status column
df_customer.rename(columns = {'fam_status' : 'family_status'}, inplace = True)

In [16]:
# Renaming first name column
df_customer.rename(columns = {'First Name' : 'first_name'}, inplace = True)

In [17]:
# Renaming Gender column
df_customer.rename(columns = {'Gender' : 'gender'}, inplace = True)

In [18]:
# Renaming STATE column
df_customer.rename(columns = {'STATE' : 'state'}, inplace = True)

In [19]:
# Renaming Age column
df_customer.rename(columns = {'Age' : 'age'}, inplace = True)

In [20]:
# Checking column names
df_customer.head()

,user_id,first_name,last_name,gender,state,age,date_joined,number_of_dependents,family_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


# Data Consistency for Customer File

## Checking for mixed data types, missing values, and duplicates

In [21]:
# Checking for mixed data types
for col in df_customer.columns.tolist():
  weird = (df_customer[[col]].map(type) != df_customer[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customer[weird]) > 0:
    print (col)

first_name


In [22]:
# Changing data type of first name column to string
df_customer['first_name'] = df_customer['first_name'].astype('str')

In [23]:
# Checking data type of first name column
df_customer['first_name'].dtype

dtype('O')

In [24]:
# Checking for missing values in customer file
df_customer.isnull().sum()

user_id                 0
first_name              0
last_name               0
gender                  0
state                   0
age                     0
date_joined             0
number_of_dependents    0
family_status           0
income                  0
dtype: int64

No missing values in customer file. 

In [25]:
# Looking for duplicates in customer data frame
df_customer[df_customer.duplicated()]

,user_id,first_name,last_name,gender,state,age,date_joined,number_of_dependents,family_status,income


No duplicates in customer file.

# Combining Orders_Products & Customer Data Sets

In [27]:
# Checking dimensions of ords_prods df 
ords_prods_merged.shape

(32404859, 25)

In [28]:
# Checking data types of ords_prods df
ords_prods_merged.dtypes

order_id                     int64
user_id                      int64
order_number                 int64
orders_day_of_week           int64
order_hour_of_day            int64
days_since_last_order      float64
product_id                   int64
add_to_cart_order            int64
reordered                    int64
_merge                    category
product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
exists                    category
price_range_loc             object
busiest_day                 object
busiest_days                object
busiest_period_of_day       object
max_order                    int64
loyalty_flag                object
avg_price_all_orders       float64
spending_flag               object
median_days_last_order     float64
frequency_flag              object
dtype: object

In [29]:
# Checking data types of customer df
df_customer.dtypes

user_id                 object
first_name              object
last_name               object
gender                  object
state                   object
age                      int64
date_joined             object
number_of_dependents     int64
family_status           object
income                   int64
dtype: object

In [30]:
# Changing data type of order_id column to string
ords_prods_merged['order_id'] = ords_prods_merged['order_id'].astype('str')

In [31]:
# Changing data type of user_id column to string
ords_prods_merged['user_id'] = ords_prods_merged['user_id'].astype('str')

In [32]:
# Dropping the merge columns in ords_prods_merged to prepare for next merge

ords_prods_merged.drop(columns=['_merge', 'exists'], inplace=True)

In [33]:
# Checking data types of ords_prods df again
ords_prods_merged.dtypes

order_id                   object
user_id                    object
order_number                int64
orders_day_of_week          int64
order_hour_of_day           int64
days_since_last_order     float64
product_id                  int64
add_to_cart_order           int64
reordered                   int64
product_name               object
aisle_id                    int64
department_id               int64
prices                    float64
price_range_loc            object
busiest_day                object
busiest_days               object
busiest_period_of_day      object
max_order                   int64
loyalty_flag               object
avg_price_all_orders      float64
spending_flag              object
median_days_last_order    float64
frequency_flag             object
dtype: object

In [36]:
# merge ords_prods df with customers df using user_id
ords_prods_all = ords_prods_merged.merge(df_customer, on=['user_id'], indicator=True)

In [38]:
# Checking dimensions of new merged file
ords_prods_all.shape

(32404859, 33)

# Exporting Merged File

In [39]:
# Exporting ords_prods_all file
ords_prods_all.to_pickle(r'/Users/suzandiab/Documents/Instacart Basket Analysis/02 Data/Prepared Data/ords_prods_all.pkl')